In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd

Using TensorFlow backend.


# 1. Read Data

In [2]:
path = "~/Desktop/kaggle/Porto/"

In [3]:
df = pd.read_csv(path+'train.csv',index_col='id',nrows=50000)

In [4]:
df.head()

target  ps_ind_01  ps_ind_02_cat  ps_ind_03  ps_ind_04_cat  ps_ind_05_cat  \
id                                                                              
7        0          2              2          5              1              0   
9        0          1              1          7              0              0   
13       0          5              4          9              1              0   
16       0          0              1          2              0              0   
17       0          0              2          0              1              0   

    ps_ind_06_bin  ps_ind_07_bin  ps_ind_08_bin  ps_ind_09_bin  \
id                                                               
7               0              1              0              0   
9               0              0              1              0   
13              0              0              1              0   
16              1              0              0              0   
17              1              0              0              0   

         ...        ps_calc_11  ps_calc_12  ps_calc_13  ps_calc_14  \
id       ...                                                         
7        ...                 9           1           5           8   
9        ...                 3           1           1           9   
13       ...                 4           2           7           7   
16       ...                 2           2           4           9   
17       ...                 3           1           1           3   

    ps_calc_15_bin  ps_calc_16_bin  ps_calc_17_bin  ps_calc_18_bin  \
id                                                                   
7                0               1               1               0   
9                0               1               1               0   
13               0               1               1               0   
16               0               0               0               0   
17               0               0               0               1   

    ps_calc_19_bin  ps_calc_20_bin  
id                                  
7                0               1  
9                1               0  
13               1               0  
16               0               0  
17               1               0  

[5 rows x 58 columns]

# 2. Get Features and Target

In [5]:
features = list(df.columns)
target = 'target'
features.remove(target)

In [6]:
X = np.array(df[features])
y = df[target]

# 3. Split train test

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 4. Define Metric

In [14]:
def gini(actual, pred):
    all = np.asarray(np.c_[ actual, pred, range(len(actual))], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
 
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

# 5. Build Model

In [45]:
model = Sequential()
model.add(Dense(12, input_dim=57, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.add(Dense(1,activation='softmax'))

In [46]:
from keras.callbacks import Callback

class gini_eval(Callback):
    def __init__(self, metric):  
        self.metric = metric
        self.maps = []
 
    def map_eval(self):    
        x_val, y_true, tmp = self.validation_data
        y_pred = self.model.predict(x_val)
        y_true = np.squeeze(y_true)
        y_pred = np.squeeze(y_pred)
        
        return self.metric(y_true,y_pred)

    def on_epoch_end(self, epoch, logs={}):

        score = self.map_eval()
        print "Gini_Coef for epoch %d is %f"%(epoch, score)
        self.maps.append(score)

In [55]:
from keras import optimizers

optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='binary_crossentropy', optimizer='adam')

metrics_callback = gini_eval(gini_normalized)

model.fit(X_train, 
          y_train, 
          epochs=20, 
          batch_size=len(y), 
          callbacks=[metrics_callback],
          validation_split=0.1, 
          #class_weight={0:1.,1:30},
          verbose=1)

Train on 31500 samples, validate on 3500 samples
Epoch 1/20
Gini_Coef for epoch 0 is 0.150185
31500/31500 [==============================] - 0s - loss: 0.1522 - val_loss: 0.1660
Epoch 2/20
Gini_Coef for epoch 1 is 0.141755
31500/31500 [==============================] - 0s - loss: 0.1542 - val_loss: 0.1652
Epoch 3/20
Gini_Coef for epoch 2 is 0.132301
31500/31500 [==============================] - 0s - loss: 0.1523 - val_loss: 0.1668
Epoch 4/20
Gini_Coef for epoch 3 is 0.126579
31500/31500 [==============================] - 0s - loss: 0.1526 - val_loss: 0.1680
Epoch 5/20
Gini_Coef for epoch 4 is 0.126293
31500/31500 [==============================] - 0s - loss: 0.1533 - val_loss: 0.1678
Epoch 6/20
Gini_Coef for epoch 5 is 0.129640
31500/31500 [==============================] - 0s - loss: 0.1531 - val_loss: 0.1667
Epoch 7/20
Gini_Coef for epoch 6 is 0.133868
31500/31500 [==============================] - 0s - loss: 0.1525 - val_loss: 0.1658
Epoch 8/20
Gini_Coef for epoch 7 is 0.139255
315

In [56]:
gini_normalized(y_test,model.predict(X_test))

0.20355477963245158